## 单字特征的解读

本教程讨论 HWDB1.0\~1.1 以及 OLHWDB1.0\~1.1，下载到个人电脑的同一目录下：

In [1]:
# CASI 数据集所在根目录
root = 'D:/datasets/OCR/CASIA/data'

载入本教程需要使用的包：

In [2]:
import struct
from pathlib import Path
from zipfile import ZipFile
import numpy as np
import tables as tb

`Path` 更加友好的管理文件的路径：

In [3]:
root = Path(root)
# 查看 root 的全部文件
[name.parts[-1] for name in root.iterdir()]

['HWDB1.0trn.zip',
 'HWDB1.0trn_gnt.zip',
 'HWDB1.0tst.zip',
 'HWDB1.0tst_gnt.zip',
 'HWDB1.1trn.zip',
 'HWDB1.1trn_gnt.zip',
 'HWDB1.1tst.zip',
 'HWDB1.1tst_gnt.zip',
 'OLHWDB1.0test_pot.zip',
 'OLHWDB1.0train_pot.zip',
 'OLHWDB1.0trn.zip',
 'OLHWDB1.0tst.zip',
 'OLHWDB1.1trn.zip',
 'OLHWDB1.1trn_pot.zip',
 'OLHWDB1.1tst.zip',
 'OLHWDB1.1tst_pot.zip']

每个单字的特征均以 `.mpf` 形式保存手工特征，可以看出上述文件均为压缩包，下面使用 `zipfile` 对压缩文件进行解读：

In [4]:
z = ZipFile(list(root.glob('**/HWDB1.0trn.zip'))[0])
z.namelist()[1:5] # 查看前4个人写的 MPF

['HWDB1.0trn/001.mpf',
 'HWDB1.0trn/002.mpf',
 'HWDB1.0trn/003.mpf',
 'HWDB1.0trn/004.mpf']

载入 MPF 的解码器：MPFDecoder

In [5]:
import sys
sys.path.append("../loader")
from casia.feature import MPFDecoder, zipfile2bunch

### 将 MPF 转换为 bunch

In [6]:
zip_name = list(root.glob('**/HWDB1.0trn.zip'))[0]
mb = zipfile2bunch(zip_name)

### 将 bunch 转换为 JSON

In [7]:
from utils.dataset import bunch2json, json2bunch

In [8]:
data_dir = Path('data')
if not data_dir.exists(): # 如果不存在
    data_dir.mkdir() # 创建目录

In [9]:
%%time
json_path = 'data/features.json'
bunch2json(mb, json_path)

Wall time: 1.14 s


In [10]:
%%time
# 再次载入数据
mpf_bunch = json2bunch(json_path)

Wall time: 1.07 s


### 将 bunch 转换为 HDF5

In [11]:
def bunch2hdf(bunch, save_path):
    '''将 bunch 转换为 HDF5'''
    filters = tb.Filters(complevel=7, shuffle=False)  # 过滤信息，用于压缩文件
    h = tb.open_file(save_path, 'w', filters=filters, title='Xinet\'s dataset')
    for name in bunch:  # 生成数据集"头"
        _name = name.replace('/', '__')
        _name = _name.replace('.', '_')
        h.create_group('/', name=_name, filters=filters)
        h.create_array(f"/{_name}", 'text',
                       bunch[name]['text'].encode())
        features = bunch[name]['dataset']
        h.create_array(f"/{_name}", 'labels',
                       " ".join([l for l in features.index]).encode())
        h.create_array(f"/{_name}", 'features', features.values)
    h.close()  # 防止资源泄露

In [12]:
%%time
hdf_path = 'data/features.h5'
bunch2hdf(mpf_bunch, hdf_path)

Wall time: 2.29 s


In [13]:
%%time
h5 = tb.open_file(hdf_path)

Wall time: 996 µs


In [14]:
# 获取某个 mpf 的特征矩阵的 shape
h5.root.HWDB1_0trn__001_mpf.features[:].shape

(3728, 512)

In [15]:
# 获取某个 mpf 的特征介绍
h5.root.HWDB1_0trn__001_mpf.text.read()

b'Character features extracted from grayscale images. #ftrtype=ncg, #norm=ldi, #aspect=4, #dirn=8, #zone=8, #zstep=8, #fstep=8, $deslant=0, $smooth=0, $nmdir=0, $multisc=0'

In [16]:
# 获取某个 mpf 的标签信息
labels = h5.root.HWDB1_0trn__001_mpf.labels.read().decode()
labels = np.array(labels.split(' '))
labels

array(['扼', '遏', '鄂', ..., '娥', '恶', '厄'], dtype='<U1')

### 测试 JSON 与 HDF5 的文件大小

In [17]:
import os
from sys import getsizeof


print(
    f"JSON Python 对象占用空间大小为 {getsizeof(mpf_bunch)/1e3} kB, 文件大小为 {os.path.getsize(json_path)/1e9} G")
print(
    f"HDF5 Python 对象占用空间大小为 {getsizeof(h5)} B, 文件大小为 {os.path.getsize(hdf_path)/1e9} G")

JSON Python 对象占用空间大小为 9.32 kB, 文件大小为 0.65487944 G
HDF5 Python 对象占用空间大小为 80 B, 文件大小为 0.644292324 G


In [18]:
h5.close()  # 关闭

## 打包多个 zip 文件

In [19]:
zip_gnt_names = set(root.glob('*gnt*.zip')) # GNT 名称列表
zip_pot_names = set(root.glob('*pot*.zip')) # POT名称列表
# MPF 名称列表
zip_mpf_names = set(root.iterdir()) - zip_pot_names - zip_gnt_names
zip_mpf_names

{WindowsPath('D:/datasets/OCR/CASIA/data/HWDB1.0trn.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/HWDB1.0tst.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/HWDB1.1trn.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/HWDB1.1tst.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/OLHWDB1.0trn.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/OLHWDB1.0tst.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/OLHWDB1.1trn.zip'),
 WindowsPath('D:/datasets/OCR/CASIA/data/OLHWDB1.1tst.zip')}

In [20]:
mpf_bunch = {}
for mpf_name in zip_mpf_names:
    mpf_bunch.update(zipfile2bunch(mpf_name))

保存为 JSON

In [21]:
%%time
json_path = 'data/features.json'
bunch2json(mpf_bunch, json_path)

Wall time: 7.49 s


保存为 HDF5

In [22]:
%%time
hdf_path = 'data/features.h5'
bunch2hdf(mpf_bunch, hdf_path)

Wall time: 12.3 s


载入 JSON

In [23]:
%%time
mpf_bunch = json2bunch(json_path)

Wall time: 9.35 s


载入 HDF5

In [24]:
%%time
h5 = tb.open_file(hdf_path)

Wall time: 4 ms


### 再次测试文件大小

In [25]:
from sys import getsizeof


print(
    f"JSON Python 对象占用空间大小为 {getsizeof(mpf_bunch)/1e3} kB, 文件大小为 {Path(json_path).stat().st_size/1e9} G")
print(
    f"HDF5 Python 对象占用空间大小为 {getsizeof(h5)} B, 文件大小为 {Path(hdf_path).stat().st_size/1e9} G")

JSON Python 对象占用空间大小为 73.824 kB, 文件大小为 2.82091995 G
HDF5 Python 对象占用空间大小为 80 B, 文件大小为 2.775284889 G


从上述的展示可以看出 HDF5 优于 JSON 与 ZipFile，所以下面仅仅考虑 HDF5 文件。

## 解析 features.h5

In [26]:
h5.get_filesize() # 获取文件大小

2775284889

In [27]:
nodes = h5.list_nodes('/')  # 列出所有 MPF 数据

In [28]:
nodes[0]

/HWDB1_0trn__001_mpf (Group) ''
  children := ['features' (Array), 'labels' (Array), 'text' (Array)]

In [29]:
len(nodes)  # 统计 MPF 个数

1440

In [30]:
data_iter = h5.iter_nodes('/') # 所有 MPF 数据以迭代器的方式使用

In [31]:
next(data_iter) # 取出一个 MPF

/HWDB1_0trn__001_mpf (Group) ''
  children := ['features' (Array), 'labels' (Array), 'text' (Array)]

### 获取 MPF 的特征矩阵与标签

In [32]:
mpf_name = 'HWDB1_0trn__007_mpf'
# 依据 MPF 的名称获取 MPF
mpf = h5.get_node('/', mpf_name)
mpf

/HWDB1_0trn__007_mpf (Group) ''
  children := ['features' (Array), 'labels' (Array), 'text' (Array)]

In [33]:
def get_features(mpf):
    '''获取 MPF 的特征矩阵'''
    return mpf.features[:]


def get_labels(mpf):
    '''获取 MPF 的标签数组'''
    labels_str = mpf.labels.read().decode()
    return np.array(labels_str.split(' '))

In [34]:
features = get_features(mpf)  # 获取特征矩阵
labels = get_labels(mpf)   # 获取标签
h5.close()

## MPF 迭代器

依据特征矩阵与标签函数，定义了 MPF 迭代器，获取方式：

In [35]:
class CASIAFeature:
    def __init__(self, hdf_path):
        '''casia 数据 MPF 特征处理工具'''
        self.h5 = tb.open_file(hdf_path)

    def _features(self, mpf):
        '''获取 MPF 的特征矩阵'''
        return mpf.features[:]

    def _labels(self, mpf):
        '''获取 MPF 的标签数组'''
        labels_str = mpf.labels.read().decode()
        return np.array(labels_str.split(' '))

    def __iter__(self):
        '''返回 (features, labels)'''
        for mpf in self.h5.iter_nodes('/'):
            yield self._features(mpf), self._labels(mpf)

### MPF 迭代器的使用方法

In [36]:
mpf_iter = CASIAFeature(hdf_path)
# 以迭代器的方式获取数据
for features, labels in mpf_iter:
    print(features.shape, labels.shape)
    break

(3728, 512) (3728,)


In [40]:
h5

<closed File>

### 为了将 CASIA 划分为训练集与测试集，需要重新打包

重启 Kernel

In [1]:
import sys
sys.path.append("../loader")

from casia.feature import CASIA

In [2]:
# CASI 数据集所在根目录
root = 'D:/datasets/OCR/CASIA/data'
save_path = 'data/features.h5'

self = CASIA(root)  # 该类实现数据集的划分
self.bunch2hdf(save_path)

In [4]:
%%time
# 载入 HDF5
import tables as tb
h5 = tb.open_file(save_path)

Wall time: 2.98 ms


In [5]:
h5.root

/ (RootGroup) "Xinet's casia dataset"
  children := ['test' (Group), 'train' (Group)]

In [6]:
from casia.feature import CASIAFeature

mpf_dataset = CASIAFeature(save_path)
# 以测试集的迭代器的方式获取数据
for features, labels in mpf_dataset.test_iter():
    print(features.shape, labels.shape)
    break

(3726, 512) (3726,)
